In [1]:
import tensorflow as tf
import numpy as np
import gym
import os
import imageio
from skimage.transform import resize
import random
import time

In [2]:
#remove deprication msgs for .layers
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [3]:
Train = True
ENV_NAME = 'BreakoutDeterministric-v4'

In [4]:
env = gym.make('BreakoutDeterministic-v4')

ACTION_SIZE = env.env.action_space.n
possible_actions = env.unwrapped.get_action_meanings()

print("size of frame: ", env.observation_space)
print("number of actions: ", ACTION_SIZE)
print("actions: ", possible_actions)

size of frame:  Box(210, 160, 3)
number of actions:  4
actions:  ['NOOP', 'FIRE', 'RIGHT', 'LEFT']


Preprocess image:

In [5]:
class FrameProcessor(object):
    def __init__(self, frame_height=84, frame_width=84):
        self.frame_height = frame_height
        self.frame_width = frame_width
        self.frame = tf.placeholder(shape=[210,160,3], dtype=tf.uint8)
        self.processed = tf.image.rgb_to_grayscale(self.frame)
        self.processed = tf.image.crop_to_bounding_box(self.processed,34, 0, 160, 160)
        self.processed = tf.image.resize_images(self.processed, [self.frame_height, frame_width],
                                                method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    def __call__(self, session, frame):
        return session.run(self.processed, feed_dict={self.frame:frame})

In [6]:
class DDQN(object):
    def __init__(self, n_actions, hidden=1024, learning_rate=0.00001,
                 frame_height=84, frame_width=84, agent_history_length=4):
        self.n_actions = n_actions
        self.hidden = hidden
        self.learning_rate = learning_rate
        self.frame_height = frame_height
        self.frame_width = frame_width
        self.agent_history_length = agent_history_length
        
        self.input = tf.placeholder(shape=[None, self.frame_height,
                                           self.frame_width, self.agent_history_length], dtype=tf.float32)
        self.inputscaled = self.input/255
        
        #CONV LAYERS:
        
        #CONV 1:
        self.conv1 = tf.layers.conv2d(inputs=self.inputscaled, filters=32, kernel_size=[8, 8], strides=4,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                     padding="valid", activation=tf.nn.relu, use_bias=False, name="conv1")
        
        #CONV 2:
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64, kernel_size=[4, 4], strides=2,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                     padding="valid", activation=tf.nn.relu, use_bias=False, name="conv2")
        
        #CONV 3:
        self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=64, kernel_size=[3, 3], strides=1,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                     padding="valid", activation=tf.nn.relu, use_bias=False, name="conv3")
        
        #CONV 4:
        self.conv4 = tf.layers.conv2d(inputs = self.conv3, filters=hidden, kernel_size=[7, 7], strides=1,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                     padding="valid", activation=tf.nn.relu, use_bias=False, name="conv4")
        
        #Split into two channels:
        self.valuestream, self.advantagestream = tf.split(self.conv4, 2, 3)
        
        self.valuestream = tf.layers.flatten(self.valuestream)
        self.advantagestream = tf.layers.flatten(self.advantagestream)
        
        self.advantage = tf.layers.dense(inputs=self.advantagestream, units=self.n_actions,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2), name="advantage")
        
        self.value = tf.layers.dense(inputs=self.valuestream, units=1,
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2), name="value")

        
        #Combine the two:
        self.q_values = self.value + tf.subtract(self.advantage, tf.reduce_mean(self.advantage, axis=1, keepdims=True))
        self.best_action = tf.argmax(self.q_values, 1)
        
        #target q:
        self.target_q = tf.placeholder(shape=[None], dtype=tf.float32)
        
        #action we took:
        self.action = tf.placeholder(shape=[None], dtype=tf.int32)
        
        #Q value of the action above:
        self.Q = tf.reduce_sum(tf.multiply(self.q_values, tf.one_hot(self.action,
                                                                     self.n_actions, dtype=tf.float32)), axis=1)
        
        
        self.loss = tf.reduce_mean(tf.losses.huber_loss(labels=self.target_q, predictions= self.Q))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.update = self.optimizer.minimize(self.loss)
        

In [7]:
class ActionGetter:
    
    def __init__(self, DQN, n_actions, eps_initial=1, eps_final=0.1, eps_final_frame=0.01,
                 eps_evaluation=0.0, eps_annealing_frames=1000000,
                 replay_memory_start_size=50000, max_frames = 25000000):
        self.DQN = DQN
        self.n_actions = n_actions
        self.eps_initial = eps_initial
        self.eps_final = eps_final
        self.eps_final_frame = eps_final_frame
        self.eps_evaluation = eps_evaluation
        self.eps_annealing_frames = eps_annealing_frames
        self.replay_memory_start_size = replay_memory_start_size
        self.max_frames = max_frames
        
        self.slope = -(self.eps_initial - self.eps_final)/self.eps_annealing_frames
        self.intercept = self.eps_initial - self.slope*self.replay_memory_start_size
        self.slope_2 = -(self.eps_final - self.eps_final_frame)/(self.max_frames - self.eps_annealing_frames - self.replay_memory_start_size)
        self.intercept_2 = self.eps_final_frame - self.slope_2*self.max_frames

    def get_action(self, session, frame_number, state, evaluation=False):
        
        if evaluation:
            eps = self.eps_evaluation
        elif frame_number < self.replay_memory_start_size:
            eps = self.eps_initial
        elif frame_number >= self.replay_memory_start_size and frame_number < self.replay_memory_start_size + self.eps_annealing_frames:
            eps = self.slope*frame_number + self.intercept
        elif frame_number >= self.replay_memory_start_size + self.eps_annealing_frames:
            eps = self.slope_2*frame_number + self.intercept_2
        
        if np.random.rand(1) < eps:
            return np.random.randint(0, self.n_actions)
        return session.run(self.DQN.best_action, feed_dict={self.DQN.input:[state]})[0]

In [8]:
class ReplayMemory(object):
    
    def __init__(self, size=1000000, frame_height=84, frame_width= 84, agent_history_length=4, batch_size=32):
        self.size = size
        self.frame_height = frame_height
        self.frame_width = frame_width
        self.agent_history_length = agent_history_length
        self.batch_size = batch_size
        self.count = 0
        self.current= 0
        
        #where the transitions will be stored:
        self.actions = np.empty(self.size, dtype=np.int32)
        self.rewards = np.empty(self.size, dtype=np.float32)
        self.frames = np.empty((self.size, self.frame_height, self.frame_width), dtype=np.uint8)
        self.terminal_flags = np.empty(self.size, dtype=np.bool)
        
        #memory for the minibatch:
        self.states = np.empty((self.batch_size, self.agent_history_length,
                                self.frame_height, self.frame_width), dtype=np.uint8)
        self.new_states = np.empty((self.batch_size, self.agent_history_length,
                                    self.frame_height, self.frame_width), dtype=np.uint8)
        self.indices = np.empty(self.batch_size, dtype=np.int32)
        
    def add_experience(self, action, frame, reward, terminal):
        
        if frame.shape != (self.frame_height, self.frame_width):
            raise ValueError("Dimensions of frame do not match 84x84")
            
        #add the experience:
        self.actions[self.current] = action
        self.frames[self.current, ...] = frame
        self.rewards[self.current] = reward
        self.terminal_flags[self.current] = terminal
        self.count = max(self.count, self.current+1)
        self.current = (self.current + 1) % self.size # if we reach the limit we start overriding the first ones
        
    def _get_state(self, index):
        if self.count is 0:
            raise ValueError("the memory is empty")
        if index < 3:
            raise ValueError("index must be at least 3")
        return self.frames[index-self.agent_history_length+1:index+1, ...] #get the 4 frames that represent this state
            
    def _get_valid_indices(self):
        for i in range(self.batch_size):
            while True:
                index = random.randint(self.agent_history_length, self.count - 1)
                if index < self.agent_history_length: # index cannot be smalled than 4
                    continue
                if index >= self.current and index - self.history_length <= self.current: # there should be atleast 4 frames to get after the state
                    continue
                if self.terminal_flags[index - self.agent_history_length:index].any(): #if there is a terminal flag active, that means that in those four frame the agent died => we do not want to take them as a state
                    continue
                break
            self.indices[i] = index
        
    def get_minibatch(self):
            
        if self.count < self.agent_history_length:
            raise ValueError("not enough memories to get a minibatch")
                
        self._get_valid_indices()
            
        for i, idx in enumerate(self.indices):
            self.states[i] = self._get_state(idx - 1)
            self.new_states[i] = self._get_state(idx)
                
        return np.transpose(self.states, axes=(0, 2, 3, 1)), self.actions[self.indices],self.rewards[self.indices], np.transpose(self.new_states, axes=(0, 2, 3, 1)), self.terminal_flags[self.indices]
    

In [9]:
def learn(session, replay_memory, main_dqn, target_dqn, batch_size, gamma):
    
    states, actions, rewards, new_states, terminal_flags = replay_memory.get_minibatch()
    
    arg_q_max = session.run(main_dqn.best_action, feed_dict={main_dqn.input:new_states})
    
    q_vals = session.run(target_dqn.q_values, feed_dict={target_dqn.input:new_states})
    double_q = q_vals[range(batch_size), arg_q_max]
    
    target_q = rewards + (gamma*double_q * (1-terminal_flags))
    
    loss, _= session.run([main_dqn.loss, main_dqn.update]
                         , feed_dict={main_dqn.input:states, main_dqn.target_q:target_q, main_dqn.action:actions})
    
    return loss

In [10]:
class TargetNetworkUpdater(object):
    def __init__(self, main_dqn_vars, target_dqn_vars):
        self.main_dqn_vars = main_dqn_vars
        self.target_dqn_vars = target_dqn_vars
        
    def _update_target_vars(self):
        update_ops = []
        for i, var in enumerate(self.main_dqn_vars):
            copy_op = self.target_dqn_vars[i].assign(var.value())
            update_ops.append(copy_op)
        return update_ops
    
    def __call__(self, sess):
        update_ops = self._update_target_vars()
        for copy_op in update_ops:
            sess.run(copy_op)
            

In [11]:
class Atari(object):
    
    def __init__(self, envName, no_op_steps=10, agent_history_length=4):
        self.env = gym.make(envName)
        self.process_frame = FrameProcessor()
        self.state = None
        self.last_lives = 0
        self.no_op_steps = no_op_steps
        self.agent_history_length = agent_history_length
        
    def reset(self, sess, evaluation=False):
        frame = self.env.reset()
        self.last_lives = 0
        terminal_life_lost = True
        
        if evaluation:
            for _ in range(random.randint(1, self.no_op_steps)):
                frame, _, _, _ = self.env.step(1) #fire the ball (only the first time it works)
                
        processed_frame = self.process_frame(sess, frame)
        self.state = np.repeat(processed_frame, self.agent_history_length, axis=2)
        
        return terminal_life_lost
    
    def step(self, sess, action):
        
        new_frame, reward, terminal, info = self.env.step(action)
        
        if info['ale.lives'] < self.last_lives:
            terminal_life_lost = True
        else:
            terminal_life_lost = terminal
        self.last_lives = info['ale.lives']
        
        processed_new_frame = self.process_frame(sess, new_frame)
        new_state = np.append(self.state[:, :, 1:], processed_new_frame, axis=2)
        self.state = new_state
        
        return processed_new_frame, reward, terminal, terminal_life_lost, new_frame
        

In [12]:
#clip the rewards to be -1, 0, 1. Proved to be benefitial for Breakout
def clip_reward(reward):
    if reward > 0:
        return 1
    elif reward == 0:
        return 0
    else:
        return -1

In [13]:
tf.reset_default_graph()
#define the hyper-params:

ENV_NAME = 'BreakoutDeterministic-v4'

MAX_EPISODE_LENGTH = 18000
EVAL_FREQUENCY = 200000
EVAL_STEPS = 10000
TARGET_NETWORK_UPDATE_FREQ = 10000

DISCOUNT_FACTOR = 0.99 # Gamma in the Q learning equation
REPLAY_MEMORY_START_SIZE = 50000 # random actions at the begining

MAX_FRAMES = 10000000
MEMORY_SIZE = 1000000
NO_OP_STEPS = 10

UPDATE_FREQ = 4
HIDDEN = 1024

LEARNING_RATE = 0.00001

BS = 32

#IMPORTANT FOR TRAINING OR TESTING:
PATH = "output/"
SUMMARIES = "summaries"
RUNID = "run1"
PATH = PATH + RUNID 
TRAIN = False

os.makedirs(PATH, exist_ok=True)
os.makedirs(os.path.join(SUMMARIES, RUNID), exist_ok=True)
SUMM_WRITER = tf.summary.FileWriter(os.path.join(SUMMARIES, RUNID))

In [14]:
atari = Atari(ENV_NAME, NO_OP_STEPS)

#create main DQN:
with tf.variable_scope('mainDQN'):
    MAIN_DQN = DDQN(atari.env.action_space.n, HIDDEN, LEARNING_RATE)

#create target DQN:
with tf.variable_scope('targetDQN'):
    TARGET_DQN = DDQN(atari.env.action_space.n, HIDDEN)
    
init= tf.global_variables_initializer()
saver = tf.train.Saver()
    
    
MAIN_DQN_VARS = tf.trainable_variables(scope='mainDQN')
TARGET_DQN_VARS = tf.trainable_variables(scope='targetDQN')

In [15]:
#Tensorboard loss and reward summary:

with tf.name_scope('Performace'):
    LOSS_PH = tf.placeholder(tf.float32, shape=None, name="loss_summary")
    LOSS_SUMMARY = tf.summary.scalar("Loss", LOSS_PH)
    REWARD_PH = tf.placeholder(tf.float32, shape=None, name="reward_summary")
    REWARD_SUMMARY = tf.summary.scalar("reward", REWARD_PH)

PERFORMANCE_SUMMARIES = tf.summary.merge([LOSS_SUMMARY, REWARD_SUMMARY])

In [16]:
def train():
    my_replay_memory = ReplayMemory(size=MEMORY_SIZE, batch_size=BS)   # (★)
    update_networks = TargetNetworkUpdater(MAIN_DQN_VARS, TARGET_DQN_VARS)
    
    action_getter = ActionGetter(MAIN_DQN, atari.env.action_space.n, 
                                 replay_memory_start_size=REPLAY_MEMORY_START_SIZE, 
                                 max_frames=MAX_FRAMES)
    
    with tf.Session() as sess:
        sess.run(init)
        
        frame_number = 0
        rewards = []
        loss_list = []
        
        while frame_number < MAX_FRAMES:
            #training:
            epoch_frame = 0
            print(frame_number)
            while epoch_frame < EVAL_FREQUENCY:
                
                terminal_life_lost = atari.reset(sess)
                episode_reward_sum = 0
                
                for _ in range(MAX_EPISODE_LENGTH):
                    
                    #take the action:
                    action = action_getter.get_action(sess, frame_number, atari.state)
                    
                    processed_new_frame, reward, terminal, terminal_life_lost, _ = atari.step(sess, action)
                    
                    frame_number += 1
                    epoch_frame += 1
                    episode_reward_sum += reward
                    
                    clipped_reward = clip_reward(reward)
                    
                    my_replay_memory.add_experience(action=action, frame=processed_new_frame[:, :, 0], 
                                                    reward=clipped_reward, terminal=terminal_life_lost)
                    
                    if frame_number % UPDATE_FREQ == 0 and frame_number > REPLAY_MEMORY_START_SIZE:
                        loss = learn(sess, my_replay_memory, MAIN_DQN, TARGET_DQN, BS, gamma = DISCOUNT_FACTOR)
                        loss_list.append(loss)
                    
                    if frame_number % TARGET_NETWORK_UPDATE_FREQ == 0 and frame_number > REPLAY_MEMORY_START_SIZE:
                        update_networks(sess)

                    if terminal:  
                        terminal = False
                        break

                rewards.append(episode_reward_sum)

                if len(rewards) % 10 == 0:
                    # summaries for tensorboard:
                    if frame_number > REPLAY_MEMORY_START_SIZE:
                        summ = sess.run(PERFORMANCE_SUMMARIES, 
                                        feed_dict={LOSS_PH:np.mean(loss_list), 
                                                    REWARD_PH:np.mean(rewards[-100:])})

                        SUMM_WRITER.add_summary(summ, frame_number)
                        loss_list = []

                    print(len(rewards), frame_number, np.mean(rewards[-100:]))
                    with open('rewards.dat', 'a') as reward_file:
                        print(len(rewards), frame_number, 
                              np.mean(rewards[-100:]), file=reward_file)

            #evaluation:
            
            terminal = True
            eval_rewards = []
            evaluate_frame_number = 0
            
            for _ in range(EVAL_STEPS):
                if terminal:
                    terminal_life_lost = atari.reset(sess, evaluation=True)
                    episode_reward_sum = 0
                    terminal = False
                    
                action = 1 if terminal_life_lost else action_getter.get_action(sess, frame_number, atari.state,
                                                                               evaluation = True)
                
                processed_new_frame, reward, terminal, terminal_life_lost, new_frame = atari.step(sess, action)
                evaluate_frame_number += 1
                episode_reward_sum += reward
                
                if terminal:
                    eval_rewards.append(episode_reward_sum)
                
            print("Evaluation score:\n", np.mean(eval_rewards))
            
            #save network params:
            saver.save(sess, PATH+"/my_model", global_step=frame_number)
            
            with open("rewardsEval.dat", "a") as eval_reward_file:
                 print(frame_number, np.mean(eval_rewards), file=eval_reward_file)
                
                

In [17]:
if TRAIN:
    train()

In [18]:
if not TRAIN:
    action_getter = ActionGetter(MAIN_DQN, atari.env.action_space.n, 
                                 replay_memory_start_size=REPLAY_MEMORY_START_SIZE, 
                                 max_frames=MAX_FRAMES)
    
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph("trained/my_model-10039387.meta")
        saver.restore(sess,tf.train.latest_checkpoint("trained"))
        terminal_life_lost = atari.reset(sess, evaluation = True)
        episode_reward_sum = 0
        while True:
            atari.env.render()
            #time.sleep(0.02)
            
            action = 1 if terminal_life_lost else action_getter.get_action(sess, 0, atari.state, evaluation = True)
                
            processed_new_frame, reward, terminal, terminal_life_lost, new_frame = atari.step(sess ,action)
            episode_reward_sum += reward
            if terminal == True:
                break
        
        atari.env.close()
        print("total reward: {}".format(episode_reward_sum))

INFO:tensorflow:Restoring parameters from trained/my_model-10039387
total reward: 393.0
